Вы обнаружили NaN и None
В таблице из предыдущего урока вместо источников трафика встречаются значения None, а вместо зашифрованных электронных адресов — NaN. Такие значения указывают, что данных нет. NaN замещает отсутствующее в ячейке число и принадлежит к типу float, поэтому с ним можно проводить математические операции. None относят к нечисловому типу NoneType, и математические операции с ним неосуществимы.
Обнаружив строки с None и NaN, не поддавайтесь искушению немедленно их удалить. Иногда пропущенные значения можно взять у разработчиков, ответственных за выгрузку данных. Кроме того, с разными типами пропущенных значений борются по-своему — этому вам и предстоит научиться.
Вызвав к столбцу source метод value_counts(), который возвращает уникальные значения и количество их упоминаний, определим, сколько раз источник трафика был пропущен.
import pandas as pd
logs = pd.read_csv('logs.csv')
print(logs['source'].value_counts())
>>>
other      133834
context     52032
email       12279
None         1674
undef         181
Name: source, dtype: int64
Источник трафика пропущен в 1674 случаях. Это менее 1% от всех значений датафрейма logs, в котором 200 000 строк. Меньше процента — это немного, и в большинстве задач удаление такой доли строк не повлияет на результаты. Но что, если все пропуски сделаны на месте значений 'email'? Тогда, удалив строки с None, мы лишим себя почти 1/7 данных: 1674 / 12279 = 13,6%. Это повлияет на метрики email-рассылок и выводы исследования.
image
Методом isna() найдём все строки с пропусками в столбце email и просмотрим первые пять.
print(logs[logs['email'].isna()].head())
      user_id   source email  purchase
0  7141786820    other   NaN         0
2  1914055396  context   NaN         0
3  4099355752    other   NaN         0
4  6032477554  context   NaN         1
5  5872473344    other   NaN         0
Этот код похож на фильтр. Выражение logs['email'].isna() проверяет все строки в столбце email и оставляет только те, в которых есть NaN. Метод head() выводит на экран первые пять строк.
Чтобы сосчитать строки без email-адресов, вызовем функцию len().
print(len(logs[logs['email'].isna()]))
186047
В 186 047 строках из 200 000 электронный адрес пропущен. Вполне объяснимо: пользователи регистрируются на сайте неохотно.
Тем не менее, пропуски нельзя оставлять без внимания: группировка данных с NaN может привести к некорректным результатам анализа. Удостоверимся на примере присуждения кубка в школе волшебников Хогвартс.
Ученики в течение года зарабатывают очки для своих факультетов: Гриффиндора, Слизерина, Пуффендуя и Когтеврана. Кубок школы предназначен факультету, набравшему наибольшее количество баллов. Цена ошибки велика.
Изучим таблицу с распределением баллов по факультетам.
import pandas as pd
hogwarts_points = pd.read_csv('/datasets/hogwarts_points.csv')

print(hogwarts_points)
>>>

faculty_name      student_name  points
0   Гриффиндор      Гарри Поттер      15
1     Слизерин      Драко Малфой      28
2    Когтевран    Полумна Лавгуд      10
3          NaN         Рон Уизли       5
4    Пуффендуй    Седрик Диггори      17
5   Гриффиндор  Невилл Долгопупс      12
Методом sum() посчитаем, сколько всего баллов заработали ученики:
print(hogwarts_points['points'].sum())
>>>

87
Уточним, сколько очков у каждого факультета. Методом groupby() cформируем таблицу с указанием суммы баллов для каждого значения столбца 'faculty_name':
print(hogwarts_points.groupby('faculty_name')['points'].sum())
>>>

 faculty_name
Гриффиндор    27
Когтевран     10
Пуффендуй     17
Слизерин      28
Name: points, dtype: int64
Сложим баллы всех факультетов методом sum():
print(hogwarts_points.groupby('faculty_name')['points'].sum().sum())
>>>

82
Количество заработанных учениками баллов (87) не равно количеству очков, начисленных факультетам (82). Всё потому, что у Рона Уизли вместо названия факультета стоит NaN, и при группировке по факультетам баллы Рона не были учтены.
Определим победителя. Метод groupby() заменяет стандартные числовые индексы на значения столбца, по которому выполняется группировка, поэтому теперь индексы — это названия факультетов. Вызовем метод idxmax(), чтобы получить индекс элемента — то есть факультет — с наибольшим численным значением.
print('Кубок получает', hogwarts_points.groupby('faculty_name')['points'].sum().idxmax())
>>> Кубок получает Слизерин
Без учёта баллов Рона кубок школы получит Слизерин.
Чтобы восстановить справедливость в Хогвартсе, нам нужно научиться заполнять пропуски в данных методом fillna(). Он заменяет все значения NaN на одно и то же новое значение, которое передается как аргумент функции. Давайте заменим все пропущенные email-адреса в таблице logs.csv на тестовый email: example@example.com. Для этого нужно вызвать метод fillna() у таблицы logs и передать в качестве аргумента строку example@example.com. А результат потом сохранить обратно в переменную logs.
import pandas as pd
logs = pd.read_csv('/datasets/logs.csv')

logs = logs.fillna('example@example.com')
Теперь посмотрим, как поменялась таблица.
print(logs.head())
user_id   source                email  purchase
0  7141786820    other  example@example.com         0
1  5644686960    email           c129aa540a         0
2  1914055396  context  example@example.com         0
3  4099355752    other  example@example.com         0
4  6032477554  context  example@example.com         1
Успех! Теперь в таблице кругом одни example-ы. Конечно, в реальной жизни не стоит заменять пропущенные email-адреса на вымышленные, да ещё и одинаковые. В последующих уроках в научитесь делать более тонкие замены, а пока давайте поупражняемся с новыми функциями.

In [55]:
import pandas as pd

1.
Восстановите справедливость: верните Рону его факультет, заменив значение NaN на 'Гриффиндор' методом fillna(). Выведите на экран таблицу hogwarts_points.

In [56]:
hogwarts_points = pd.read_csv('datasets/hogwarts_points.csv')
hogwarts_points['faculty_name'] = hogwarts_points['faculty_name'].fillna('Гриффиндор')
print(hogwarts_points)

  faculty_name      student_name  points
0   Гриффиндор      Гарри Поттер      15
1     Слизерин      Драко Малфой      28
2    Когтевран    Полумна Лавгуд      10
3   Гриффиндор         Рон Уизли       5
4    Пуффендуй    Седрик Диггори      17
5   Гриффиндор  Невилл Долгопупс      12



2.
Измените код, чтобы результат приобрёл такой вид:
Сумма баллов учеников: # сумма значений столбца 'points'
Сумма баллов факультетов: # сумма баллов при группировке по факультетам
Кубок получает # название факультета

In [57]:
print(f'Сумма баллов учеников:', hogwarts_points['points'].sum()) # сумма значений столбца 'points'
print('Сумма баллов факультетов:', hogwarts_points.groupby('faculty_name')['points'].sum().sum()) # сгруппируйте по столбцу 'faculty_name'
print('Кубок получает',hogwarts_points.groupby('faculty_name')['points'].sum().idxmax())

Сумма баллов учеников: 87
Сумма баллов факультетов: 87
Кубок получает Гриффиндор


3.
Применим полученные в Хогвартсе знания к реальной задаче. Возьмём источники трафика Яндекс.Маркета и рассчитаем конверсию.
Найдите количество визитов из каждого источника трафика. Для этого сгруппируйте данные из столбца с идентификаторами пользователей по столбцу с источниками. Результат сохраните в переменной visits и выведите на экран.

In [58]:
import pandas as pd

logs = pd.read_csv('datasets/logs.csv')
print(logs.head(10))
# само задание
visits = logs.groupby('source')['user_id'].count()
print(visits)

      user_id   source       email  purchase
0  7141786820    other         NaN         0
1  5644686960    email  c129aa540a         0
2  1914055396  context         NaN         0
3  4099355752    other         NaN         0
4  6032477554  context         NaN         1
5  5872473344    other         NaN         0
6  7977025176    other         NaN         0
7  3512872755    other         NaN         0
8  1827368713  context         NaN         0
9  8688870165    other         NaN         0
source
None         1674
context     52032
email       12279
other      133834
undef         181
Name: user_id, dtype: int64


4.
Посчитайте количество совершённых покупок для каждого источника трафика. Результат сохраните в переменной purchase и выведите на экран.

In [ ]:
purchase = logs.groupby('source')['purchase'].sum()
print(purchase)

5.
Посчитайте конверсию по каждому источнику трафика, результат сохраните в переменной conversion и выведите на экран.

In [ ]:
conversion = purchase/visits
print(conversion)